In [1]:
%load_ext autoreload
%autoreload 2

# Make node mapping between two versions of the tf dataset

In this notebook we map the *slot* nodes from version 0.4 (source version) to 0.7 (target version).

Basically this means that we map all slots from the source version to corresponding slots in the target version.
In the target version there are more slots, because also footnotes occupy slots there, in contrast with the
source version, where footnotes only appear inside feature values of the slot that precedes the footnote mark.

Some slots have an empty text (most of them contain some punctuation).

We do not want to be fussy about those slots.
We map them unto corresponding empty slots if possible, otherwise we map them onto the nearest
non-empty slot.

After establishing the slot mapping, we extend the mapping to all nodes in a generic way.
The code for this is already in the TF library.

In [2]:
from tf.fabric import Fabric
from tf.compose import Versions

from lib import TF_DIR

va = "0.4"
vb = "0.7"

We load the two versions of the TF data by means of the lower level `Fabric` method,
and we only load the features we need.

In [3]:
TF = {}
api = {}
E = {}
Es = {}
F = {}
Fs = {}
L = {}
T = {}
maxSlot = {}
features = {
    va: "trans",
    vb: "trans isnote",
}

In [5]:
for v in (va, vb):
    TF[v] = Fabric(locations=TF_DIR, modules=v)
    api[v] = TF[v].load(features[v])

This is Text-Fabric 8.4.7
Api reference : https://annotation.github.io/text-fabric/cheatsheet.html

35 features found and 0 ignored
  0.00s loading features ...
   |     1.62s T otype                from ~/github/Dans-labs/clariah-gm/tf/0.4
   |       14s T oslots               from ~/github/Dans-labs/clariah-gm/tf/0.4
   |     8.48s T punc                 from ~/github/Dans-labs/clariah-gm/tf/0.4
   |     3.75s T transr               from ~/github/Dans-labs/clariah-gm/tf/0.4
   |     0.63s T n                    from ~/github/Dans-labs/clariah-gm/tf/0.4
   |     7.04s T transo               from ~/github/Dans-labs/clariah-gm/tf/0.4
   |     5.65s T punco                from ~/github/Dans-labs/clariah-gm/tf/0.4
   |     0.01s T title                from ~/github/Dans-labs/clariah-gm/tf/0.4
   |     3.17s T puncr                from ~/github/Dans-labs/clariah-gm/tf/0.4
   |       11s T trans                from ~/github/Dans-labs/clariah-gm/tf/0.4
   |      |     0.37s C __levels__     

   |      |     1.21s c __sections__         WARNING:   93 x section line without containing page


   |      |     1.28s C __sections__         from otype, oslots, otext, __levUp__, __levels__, n, n, n
   |      |     0.25s C __structure__        from otype, oslots, otext, __rank__, __levUp__, n, title, n
 2m 52s All features loaded/computed - for details use loadLog()
This is Text-Fabric 8.4.7
Api reference : https://annotation.github.io/text-fabric/cheatsheet.html

41 features found and 0 ignored
  0.00s loading features ...
  9.87s All features loaded/computed - for details use loadLog()


We put the parts of the TF API that we need in the various dictionaries.

In [6]:
for v in (va, vb):
    E[v] = api[v].E
    Es[v] = api[v].Es
    F[v] = api[v].F
    Fs[v] = api[v].Fs
    L[v] = api[v].L
    T[v] = api[v].T
    maxSlot[v] = F[v].otype.maxSlot

## Make the slot mapping

We walk through the slots of the target version (0.7) and skip its footnote slots.
For each target slot we increase the slot in the source version in (0.5), and check whether
source and target slots have the same value for the `trans` feature.
If not, and one of them is empty, we skip the empty word and try the next one.
But if both are not empty and unequal, we have a real problem: a mismatch.

However, in version 0.5 we have an imperfect separation of numbers and words.
So, sometimes we have to split words.

In that case we stop, and you have to inspect what is happening.

In [7]:
def makeSlotMapOld():
    Fa = F[va]
    Fb = F[vb]
    transA = Fa.trans.v
    transB = Fb.trans.v
    isNote = Fb.isnote.v
    maxSlotA = maxSlot[va]
    maxSlotB = maxSlot[vb]

    print(
        f"""\
    Computing slotMap between:
    {va}: {maxSlotA:>8} slots,
    {vb}: {maxSlotB:>8} slots.\
"""
    )

    slotMap = {}

    good = True
    wA = 1
    emptyA = 0
    emptyB = 0

    for wB in range(1, maxSlotB + 1):
        if isNote(wB):
            continue
        textA = transA(wA) or ""
        textB = transB(wB) or ""

        if textB == "":
            if textA != "":
                emptyB += 1
                continue
        else:
            while textA == "" and wA < maxSlotA:
                wA += 1
                emptyA += 1
                textA = transA(wA) or ""

        if textA != textB:
            print("Mismatch:")
            print(f"A: {wA:>8} = `{textA}`")
            print(f"B: {wB:>8} = `{textB}`")
            good = False
            break

        if wA <= maxSlotA:
            slotMap.setdefault(wA, {})[wB] = None
            wA += 1
        else:
            if textB:
                print(f"No more slots in {va} to match slot {wB} in {vb}")
                break

    maxSlotMap = max(slotMap)
    if maxSlotMap > maxSlotA:
        print(f"maxSlot in A version {va} exceeded")
        print(f"Found {maxSlotMap}, but it should be <= {maxSlot[va]}")
        good = False

    if good:
        print(
            f"""\
slotMap succesfully created: {len(slotMap)} slots mapped.
{va}: {emptyA:>6} empty slots,
{vb}: {emptyB:>6} empty slots.\
"""
        )
    return slotMap

In [8]:
def makeSlotMap():
    Fa = F[va]
    Fb = F[vb]
    transA = Fa.trans.v
    transB = Fb.trans.v
    isNote = Fb.isnote.v
    maxSlotA = maxSlot[va]
    maxSlotB = maxSlot[vb]

    print(
        f"""\
    Computing slotMap between:
    {va}: {maxSlotA:>8} slots,
    {vb}: {maxSlotB:>8} slots.\
"""
    )

    slotMap = {}

    good = True
    wA = 1
    wB = 1

    while wB <= maxSlotB and wA <= maxSlotA:
        if isNote(wB):
            wB += 1
            continue

        textA = transA(wA) or ""
        textB = transB(wB) or ""

        if textA == textB:
            slotMap.setdefault(wA, {})[wB] = None
            wA += 1
            wB += 1

        elif textA.startswith(textB):
            slotMap.setdefault(wA, {})[wB] = None
            wB += 1
        elif textA.endswith(textB):
            wA += 1
            wB += 1

        elif textB.startswith(textA):
            slotMap.setdefault(wA, {})[wB] = None
            wA += 1
        elif textB.endswith(textA):
            slotMap.setdefault(wA, {})[wB] = None
            wA += 1
            wB += 1

        else:
            print("Mismatch:")
            print(f"A: {wA:>8} = `{textA}`")
            print(f"B: {wB:>8} = `{textB}`")
            good = False
            break

    maxSlotMap = max(slotMap)
    if maxSlotMap > maxSlotA:
        print(f"maxSlot in A version {va} exceeded")
        print(f"Found {maxSlotMap}, but it should be <= {maxSlot[va]}")
        good = False

    if good:
        print(
            f"""\
slotMap succesfully created: {len(slotMap)} slots mapped.
"""
        )
    return slotMap

In [9]:
slotMap = makeSlotMap()

    Computing slotMap between:
    0.4:  5030444 slots,
    0.7:  5316429 slots.
slotMap succesfully created: 5030444 slots mapped.



When we encounter problems, we can do a bit of checking to see what is going on.

The next function shows the line around a slot node, and can do so in both versions.

In [10]:
def show(v, n):
    lines = L[v].u(n, otype="line")
    if not lines:
        lines = L[v].u(n + 1, otype="line")
    if not lines:
        lines = L[v].u(n - 1, otype="line")
    if not lines:
        print("no such line")
        return
    line = lines[0]
    print(T[v].sectionFromNode(line))
    words = L[v].d(line, otype="word")
    print(" ".join(f"[{w}={F[v].trans.v(w)}]" for w in words))
    print(T[v].text(line))

In [11]:
show(va, 49)
show(vb, 96)

(1, None, 2)
[45=961] [46=copie] [47=5] [48=folio] [49=s]
961, copie, 5 folio's.
(1, None, 2)
[94=961] [95=copie] [96=5folio] [97=s]
961, copie, 5folio's.


## Make the complete node map

We now extend the slotMap to a full node map.

See [compose.Versions](https://annotation.github.io/text-fabric/compose/Versions.html) in the Text-Fabric documentation.

In [12]:
V = Versions(api, va, vb, slotMap)

In [13]:
V.makeVersionMapping()


**********************************************************************************************
*                                                                                            *
*       0.00s Mapping volume nodes 0.4 ==> 0.7                                               *
*                                                                                            *
**********************************************************************************************

|       0.00s Extending slot mapping 0.4 ==> 0.7 for volume nodes
|         22s 	Done
..............................................................................................
.         22s Statistics for 0.4 ==> 0.7 (volume)                                            .
..............................................................................................
|         22s 	TOTAL                          : 100.00%      13x
|         22s 	unique, imperfect              : 100.00%      13x

*********************

The result is a new feature in the 0.7 version of the dataset:

In [15]:
!ls -l ~/github/Dans-labs/clariah-gm/tf/{vb}/omap#{va}-{vb}.tf

-rw-r--r--  1 dirk  staff  44603646 Jan 30 14:58 /Users/dirk/github/Dans-labs/clariah-gm/tf/0.7/omap#0.4-0.7.tf


## Observations

1. in the vast majority of cases, a node in the source version has just one obvious counterpart in the target version
2. most cases of ambiguity arise in the node type `line`.

Maybe we can shed some light on those cases.

First we load the mapping as a TF edge feature:


We are interested in mappings between line nodes which are diagnosed as **multiple, non-perfect**.
First we ask for the list of diagnostic labels:

In [16]:
V.legend()

b = unique, perfect
d = multiple, one perfect
c = unique, imperfect
f = multiple, cleanly composed
e = multiple, non-perfect
a = not mapped


We need to inspect line nodes in version 0.6 that have label `e`:

In [17]:
diags = V.getDiagnosis(node="line", label="e")
print(type(diags))
print(len(diags))

<class 'tuple'>
10612


In [18]:
T[va].text(diags[0])

'VOOR ILE DE MAYO 25 februari 1610.'

In [19]:
V.edge[diags[0]]

{5340753: 19, 5340757: 4}

In [20]:
for (lnb, dis) in V.edge[diags[0]].items():
    print(f"dis={dis:>2} text={T[vb].text(lnb)}")

dis=19 text=VOOR ILE DE MAYO De eerste drie brieven, door Both op reis naar Indië geschrovon, wijken niet af van
dis= 4 text= 25 februari 1610.


# Explanation

In the target version footnotes occupy lines themselves. Linebreaks in footnotes now become linebreaks in the text as a whole.
So lines in the source version may become split into several parts when they have a reference to a multiline footnote.

The mapping then detects the two target lines, each of which is an imperfect target of the source line.
We cannot do much about it.

We could have made another coding decision: line breaks in footnotes are different from linebreaks in the body text.
Then we would have a good correspondence between the lines in both versions.